In [2]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from __future__ import print_function

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import stanza
import json
import re

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


stanza.download('ar')

INFO:stanza:Downloading default packages for language: ar (Arabic) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##reading the data

In [5]:
train_pos =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Big_data/ Arabic Sentiment Twitter Corpus/train_Arabic_tweets_negative_20190413.tsv',sep='\t' , names = ['label','text'])

In [6]:
train_pos

,label,text
0,neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,neg,الدودو جايه تكمل علي 💔
...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,neg,احسدك على الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔


In [7]:
train_neg =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Big_data/ Arabic Sentiment Twitter Corpus/train_Arabic_tweets_negative_20190413.tsv',sep='\t',names = ['label','text'])

In [8]:
test_neg =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Big_data/ Arabic Sentiment Twitter Corpus/test_Arabic_tweets_negative_20190413.tsv',sep='\t',names = ['label','text'])

In [9]:
test_pos =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Big_data/ Arabic Sentiment Twitter Corpus/test_Arabic_tweets_positive_20190413.tsv',sep='\t',names = ['label','text'])

In [10]:
test_neg

,label,text
0,neg,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,neg,واحد تبع النظام السوري يقول أن المخابرات السور...
2,neg,الى متى التعامل السئ للخادمات وعدم احترامهم وك...
3,neg,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,neg,تتمغط ومعها سداع 😫
...,...,...
5763,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...
5764,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...
5765,neg,انا كنت اكل الصحن بكبره 😐
5766,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...


In [30]:
nlp = stanza.Pipeline(lang='ar', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| pos       | padt    |
| lemma     | padt    |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


##concatenate positive and nagative data

In [11]:
df_train =pd.concat([train_pos,train_neg], axis=0 ,ignore_index = True)

In [12]:
df_test =pd.concat([test_pos,test_neg], axis=0 ,ignore_index = True)

In [13]:
df =pd.concat([df_train,df_test], axis=0 ,ignore_index = True)

In [14]:
df

,label,text
0,neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,neg,الدودو جايه تكمل علي 💔
...,...,...
56543,neg,النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...
56544,neg,استشهاد_الامام_كاظم_الغيظ السلام على المعذب في...
56545,neg,انا كنت اكل الصحن بكبره 😐
56546,neg,قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...


##preprocessing

In [15]:
train_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22514 entries, 0 to 22513
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   22514 non-null  object
 1   text    22514 non-null  object
dtypes: object(2)
memory usage: 351.9+ KB


#### Drop dulicates and null values


In [16]:
# Drop dulicates and null values

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22796 entries, 0 to 56546
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   22796 non-null  object
 1   text    22796 non-null  object
dtypes: object(2)
memory usage: 534.3+ KB


In [51]:
# Drop dulicates and null values
#arabic_data =[df_train , df_test]

#for data in arabic_data:
  #data.drop_duplicates(inplace=True)
  #data.dropna(inplace=True)
  #data.info()

###remove the emojis

In [17]:
def remove_emojis(data):
   emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
   return re.sub(emoj, '', data)
   return remove_emojis(data)

In [18]:
#change the datatype to string
df_1 = str(df)


In [19]:
#call the remove emojis func
df_1 = remove_emojis(df_1)

In [20]:
df_1

'      label                                               text\n0       neg  اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...\n1       neg  توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...\n2       neg  #الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...\n3       neg  نعمة المضادات الحيوية . تضع قطرةمضاد بنسلين ع...\n4       neg                             الدودو جايه تكمل علي \n...     ...                                                ...\n56534   neg                         انت من زمان بس ماش ماتحس \n56538   neg  قبل فتره عرفت ان جدي هدد خالتي بالقتل عشانه فك...\n56543   neg  النوم وانت مكسور ده احساس غبي اللي هو مش قادر ...\n56545   neg                          انا كنت اكل الصحن بكبره \n56546   neg  قولوا لي ايش تشوفوا .. مع ملاحظة التلطف لأنه ا...\n\n[22796 rows x 2 columns]'

In [21]:
df['label'].value_counts()

neg    18410
pos     4386
Name: label, dtype: int64

In [22]:
df = df.replace(r'\\n\\r',' . ',regex=True)
df['text'][0]

'اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭'

In [24]:
#df_smpl = df.sample(n=5000)
#df_smpl['label'].value_counts()

neg    4044
pos     956
Name: label, dtype: int64

In [27]:
# Splitting data
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text'],df['label'],test_size=0.2,random_state=42)

In [28]:
#Encoding the data 

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [31]:
%%time

# pre-processing
text = []

 # POS Tags to be removed , as it represent stop words here
stop_ar = {'ADP','AUX','CCONJ','PART','PRON','SCONJ','PUNCT'}

for i in list(df['text']):
    # Remove all characters other than arabic ones
  txt = re.sub('[^ء-ي]', ' ', str(i)) 
    # Apply the stanza core nlp analyzer
  doc = nlp(txt)

  txt = [word.text for sent in doc.sentences for word in sent.words if (word.upos not in stop_ar and len(word.text) > 1)]
  txt = ' '.join(txt)
  text.append(txt)

CPU times: user 21min 13s, sys: 5.42 s, total: 21min 19s
Wall time: 21min 17s


In [32]:
len(text)

22796

In [33]:
Tfidf_vect = TfidfVectorizer()
#Tfidf_vect.fit(dff_smpl['full_text'])
Tfidf_vect.fit(text)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
print (Test_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)
print(Train_X_Tfidf.shape)
print(Test_X_Tfidf.shape)

49013    ✨ ليش الحكم ماأعطىٰ قوميز بطاقة صفراء. جراء تع...
6861              وايد قاعده احاتي ماعرف اشلون بقدر انام 😢
9158                    هذا رئيس تحرير جريدة 😵 الطم يا ناس
12452                             متعمد هالسماجه للأمانه 😔
15698                                        فاصل ونواصل 💔
                               ...                        
1512     لماذا الولد اذكى من البنت ؟؟ — الله اعلم بس اي...
14155    أوف في حياتي ما سمعت صوت بالفخامة ورخامة ذا ال...
53305    حقيقي جيني اكثر وحدة اكلت خرا السنة اللي فاتت ...
51274    لماذا العنصرية لم تلغى عند البشر؟! و في هذا ال...
4383                          تفاعلكم تببن ☹ #معصيتي_راحتي
Name: text, Length: 4560, dtype: object
(18236, 45102)
(4560, 45102)


In [34]:
model = SVC(kernel='linear',probability=True)
TFO = model.fit(Train_X_Tfidf,Train_Y)

In [35]:
# Classification report

y_eval = TFO.predict(Test_X_Tfidf)
print(metrics.classification_report(Encoder.inverse_transform(Test_Y),Encoder.inverse_transform(y_eval)))


              precision    recall  f1-score   support

         neg       0.83      0.98      0.90      3705
         pos       0.62      0.16      0.25       855

    accuracy                           0.82      4560
   macro avg       0.73      0.57      0.57      4560
weighted avg       0.79      0.82      0.78      4560

